In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
import io

class ObjectDetector:
    def __init__(self):
        self.image = None
        self.processed_image = None
        
        # Buat widget
        self.upload = widgets.FileUpload(description='Upload Image', accept='image/*', multiple=False)
        self.threshold1 = widgets.IntSlider(value=50, min=0, max=255, description='Threshold 1:')
        self.threshold2 = widgets.IntSlider(value=150, min=0, max=255, description='Threshold 2:')
        self.min_area = widgets.IntSlider(value=500, min=10, max=5000, description='Min Area:')
        self.blur_size = widgets.IntSlider(value=5, min=1, max=15, step=2, description='Blur Size:')
        self.detect_button = widgets.Button(description="Detect Objects")
        self.output = widgets.Output()
        
        # Atur event handler
        self.upload.observe(self.on_upload, names='value')
        self.detect_button.on_click(self.on_detect)
        
        # Tampilkan widget
        display(widgets.VBox([
            self.upload,
            widgets.HBox([self.threshold1, self.threshold2]),
            widgets.HBox([self.min_area, self.blur_size]),
            self.detect_button,
            self.output
        ]))
    
    def on_upload(self, change):
        with self.output:
            clear_output()
            if not change['new']:
                return

            uploaded_files = self.upload.value
            if not uploaded_files:
                print("No file uploaded.")
                return

            uploaded_file = uploaded_files[0]
            image_data = uploaded_file['content']
            self.image = cv2.imdecode(np.frombuffer(image_data, np.uint8), cv2.IMREAD_COLOR)
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)

            # Tampilkan gambar asli
            plt.figure(figsize=(10, 6))
            plt.imshow(self.image)
            plt.title("Original Image")
            plt.axis('off')
            plt.show()
    
    def on_detect(self, button):
        with self.output:
            clear_output()
            if self.image is None:
                print("Please upload an image first")
                return

            # Proses gambar
            gray = cv2.cvtColor(self.image, cv2.COLOR_RGB2GRAY)
            blurred = cv2.GaussianBlur(gray, (self.blur_size.value, self.blur_size.value), 0)
            edges = cv2.Canny(blurred, self.threshold1.value, self.threshold2.value)

            # Temukan kontur
            contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Filter kontur berdasarkan area
            large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > self.min_area.value]

            # Gambar bounding box
            processed_image = self.image.copy()
            for contour in large_contours:
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(processed_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Tampilkan hasil
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

            ax1.imshow(self.image)
            ax1.set_title('Original Image')
            ax1.axis('off')

            ax2.imshow(edges, cmap='gray')
            ax2.set_title('Edge Detection')
            ax2.axis('off')

            ax3.imshow(processed_image)
            ax3.set_title(f'Detected Objects ({len(large_contours)} objects)')
            ax3.axis('off')

            plt.show()

            print(f"Detected {len(large_contours)} objects")

# Jalankan detektor
detector = ObjectDetector()
